In [1]:
# Translated to .py by Yundi Zhang
# Jan 05 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Nov  4 14:03:17 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.models import loglogit

pandas = pd.read_table("netherlands.dat")
database = db.Database("netherlands",pandas)
pd.options.display.float_format = '{:.3g}'.format


globals().update(database.variables)
from biogeme.expressions import *

exclude = sp != 0
database.remove(exclude)


#Parameters to be estimated
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
ASC_RP_CAR	 = Beta('ASC_RP_CAR',0,None,None,1)
ASC_RP_RAIL	 = Beta('ASC_RP_RAIL',0,None,None,0)
BETA_COST	 = Beta('BETA_COST',0,None,None,0)
BETA_TIME	 = Beta('BETA_TIME',0,None,None,0)

# Define here arithmetic expressions for name that are not directly available from the data
rail_time  = DefineVariable('rail_time', rail_ivtt   +  rp_rail_ovt ,database)
car_time  = DefineVariable('car_time', car_ivtt   +  rp_car_ovt ,database )

# Utilities
Car = ASC_RP_CAR + BETA_COST * car_cost + BETA_TIME * car_time
Rail = ASC_RP_RAIL + BETA_COST * rail_cost + BETA_TIME * rail_time
V = {0: Car,1: Rail}
av = {0: rp,1: rp}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,choice)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "RP-SP_NL_rp"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  # This is added back by InteractiveShellApp.init_path()


              Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
ASC_RP_RAIL   0.798     0.27    2.95  0.00315         0.275          2.9   
BETA_COST   -0.0499   0.0103   -4.85 1.21e-06        0.0107        -4.67   
BETA_TIME     -1.33    0.344   -3.86 0.000115         0.354        -3.75   

             Rob. p-value  
ASC_RP_RAIL       0.00379  
BETA_COST        3.03e-06  
BETA_TIME         0.00018  
Nbr of observations: 228
LL(0) =    -158.038
LL(beta) = -123.133
rho bar square = 0.202
Output file: RP-SP_NL_rp.html


In [2]:
pandasResults

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_RP_RAIL,0.798,0.27,2.95,0.00315,0.275,2.9,0.00379
BETA_COST,-0.0499,0.0103,-4.85,1.21e-06,0.0107,-4.67,3.03e-06
BETA_TIME,-1.33,0.344,-3.86,0.000115,0.354,-3.75,0.00018
